<a href="https://colab.research.google.com/github/kgaurav3838/BCS_Workshop_Apr_20/blob/master/_MyMTechThesisProjectTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon June 1 22:47:29 2020
Updated on Aug 27
@author: Kumar Gaurav
Institutions: EE, IIT Kanpur

Title : Traffic Vehicle Detection, Tracking and Speed Estimation [Final main code]
------------------------------------------------------------------
calculates speed of vehicles in realtime in KMPH
Added lateral distance tracking of two oppsite going vehicles
computed acceleration

key_formula =  ( speed in object plane / distance of camera from road ) = ( speed in image plane / focal length )
approx fps of the video = 30

to obtain accurate value distance to object (mm)  =   focal length (mm) * real height of the object (mm) * image height (pixels)
                                                       ----------------------------------------------------------------
                                                                 object height (pixels) * sensor height (mm)
Sw	13.2	= the sensor width of the camera (millimeters)	
FR	8.8	= the focal length of the camera (millimeters)	
H (Drone height)	100	= the flight height (meters)	
imW	4096	= the image width (pixels)	
imH	2160	= the image height (pixels)	
GSD	3.66	= Ground Sampling Distance (centimeters/pixel)	
Dw	150	= width of single image footprint on the ground (meters)	
DH	79	= height of single image footprint on the ground (meters)	

1. Lens Focal Length in millimeters - 8.8 mm
2. The sensor size
Sensor Width - 13.2 mm
Sensor Height - 8.8 mm
Usage: knnSubtractor
"""
import cv2
#from google.colab.patches import cv2_imshow # Colab cv2 imshow
import numpy as np
import random
from collections import deque
import imutils as im
import time
import datetime
import sys
#sys.path.insert(0, "/content/drive/My Drive/Colab Notebooks/")
#from align import alignImages

def make_new_color():
    return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
def make_new_veh():
    return [random.randint(1, 200)]

def rotate_Frame_Video_Single(image, angle):  # For Single Videos
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    result = result[:,50:4000]
    result1 = result.copy()
    #result[:985 ,] = 0 ;result[1170: ,] = 0
    result[:955 ,] = 0 ;result[1200: ,] = 0
    return result, result1

def rotate_Frame_Video_Stitched(image, angle):   # For Pyramid stitched videos
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result0 = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    result = np.zeros((image.shape[0]+400,image.shape[1],3), np.uint8)
    result[100:1612,50:5500] = result0[:,50:5500]
    result1 = result.copy()
    #result[:985 ,] = 0 ;result[1170: ,] = 0
    result[:751 ,] = 0 ;result[1000: ,] = 0   # 875
    return result, result1

def shadow_removal(img):
    rgb_planes = cv2.split(img)
    
    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((5,5), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)
    
    result = cv2.merge(result_planes)
    result_norm = cv2.merge(result_norm_planes)
    return result_norm
def canny_edge_detector(image): 
      
    # Convert the image color to grayscale 
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)  
      
    # Reduce noise from the image 
    blur = cv2.GaussianBlur(gray_image, (5, 5), 0)  
    canny = cv2.Canny(blur, 50, 150) 
    return canny 
def region_of_interest(image): 
    #height = image.shape[0] 
    '''polygons = np.array([ 
        [(200, height), (1100, height), (550, 250)]   # (width, height)
        ]) '''
    #polygons = np.array([[ (0, 900), (0,1250), (image.shape[1],1250), (image.shape[1], 900) ]])
    #polygons = np.array([[ (0, 1030), (0,1150), (image.shape[1],1150), (image.shape[1], 1030) ]])
    polygons = np.array([[ (0, 1100), (0,1230), (image.shape[1],1230), (image.shape[1], 1100) ]])
    mask = np.zeros_like(image) 
    # Fill poly-function deals with multiple polygon 
    cv2.fillPoly(mask, polygons, 255)  
      
    # Bitwise operation between canny image and mask image 
    masked_image = cv2.bitwise_and(image, mask)  
    return masked_image
def getTransform(image, ty):
    rows,cols = image.shape[:2]
    #pts1 = np.float32([[50,50],[200,50],[50,200]])
    #pts2 = np.float32([[10,100],[200,50],[100,250]])
    #M = cv2.getAffineTransform(pts1,pts2)
    ty = ty - cols
    M = np.float32([[1,0,0],[0,1,ty]])
    dst = cv2.warpAffine(image,M,(cols,rows))
    return dst

# Sensor Details:
GSD = 0.0366 #3.66 cm/pixels
Iw = 4096; Sw = 13.2 # in mm
value = (GSD*Iw)/Sw
f = 8.8; distance = 100 # approximated, real distance = (R/r)*(f*W/w)
ratio = distance/f
count = 0
car_count = 0
temp_new_ID = 0
temp_count = 0

#/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4
# capt.Video frames from a video
print("INFO: Video Starting")
#path = '/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4'
#path = '/content/drive/My Drive/Colab Notebooks/Colab Notebooks/Videos/2ndDrone.MP4'
pathc = '/content/drive/My Drive/Colab Notebooks/Colab Notebooks Anshu/Videos/_1stDrone_aligned.mp4'
pathw ="R:\My Files\OneDrive - IIT Kanpur\Tracking Vehicles Data and Result\Stabilized Drone Data\Stabilized Videos Data\_1stDrone_aligned.mp4"
pathw1 = "R:/My Files/OneDrive - IIT Kanpur/Video Data/StitchData/_1stDrone_NewAlign_02.mp4"  # Video_AlignPyramidStitch_117, Video_stitch_1+2_02
pathw2 = "R:/My Files/OneDrive - IIT Kanpur/Video Data/StitchData/alignVideos/Video_AlignPyramidStitch_115_01.mp4"
pathw3 = "E:\OneDrive - IIT Kanpur\Tracking Vehicles Data and Result\Stabilized Drone Data\Stabilized Videos Data\_1stDrone_aligned.mp4"
cap = cv2.VideoCapture(pathc)

print("Video Open-",cap.isOpened())
# Subtractors
sub = cv2.createBackgroundSubtractorMOG2()  # create background subtractor
mogSubtractor = cv2.bgsegm.createBackgroundSubtractorMOG(300)
mog2Subtractor = cv2.createBackgroundSubtractorMOG2() 
gmgSubtractor = cv2.bgsegm.createBackgroundSubtractorGMG(10, .8)
knnSubtractor = cv2.createBackgroundSubtractorKNN()   #Very efficient if number of foreground pixels is low
cntSubtractor = cv2.bgsegm.createBackgroundSubtractorCNT(5, True)
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,12))

centroids_list = deque([])
car_count = 0
temp1 = 0
with open("lateral_distance.csv", "w") as o:
    pass

#width = 1500 ; height = 820
#width = 2000; height = 1093
width = 2500; height = 1367
#width = 3000; height = 832
#frate = int(cap.get(cv2.CAP_PROP_FPS))
frate = 20
#***************************************************************************************************************
#path21 = 'D:/Gaurav/StitchData/VideoWrite/NewStitch3/_TrackVehicle_SingleDrone_Super_1.mp4' # Video write
#path21 = "D:/Gaurav/StitchData/VideoWrite/NewStitch3/TrackVehicle_SingleDrone_super8test.mp4"
#path21 = "E:/CodeVideoData/Ultimate Trials/TrackVehicle_SingleDrone_superTest_02.mp4"
path21 = '/content/drive/My Drive/Colab Notebooks/Videos/_Track_Traffic_Vehicles_Colab_Ultimate_Test_32.mp4'
writer = cv2.VideoWriter(path21, cv2.VideoWriter_fourcc(*'XVID'),frate, (width, height))

frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT ))
print("No. of frames:", frames)

print("INFO: Tracking is in Progress...")
frame_count = 0
start = time.time()
dist_dict = {}
first_frame = False
#cv2.namedWindow("output", cv2.WINDOW_NORMAL)

while True:

    ret, frame = cap.read()  # import image
    frame_count +=  1
    if frame_count == 3600: # 7200, 3600, 3700, 3615, 5800
      break
    if frame_count < 3100:
        continue

    if  ret:  # if there is a frame continue with code
        if first_frame:
            imgRef = frame
            first_frame = False

        #frame = alignImages(frame, imgRef)
        image, image2 = rotate_Frame_Video_Single(frame, 1.58) 

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # converts image to gray
        fg_mask = knnSubtractor.apply(gray)  #  mog2Subtractor  knnSubtractor
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        value1, th = cv2.threshold(fg_mask, 5, 255, cv2.THRESH_BINARY)
        opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=1) #erosion followed by dilation, useful in removing noises
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=3) #Dilation followed by Erosion
        dilation = cv2.dilate(closing, kernel, iterations = 1)
        #closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel, iterations=1) #Dilation followed by Erosion
        '''        
        dilation = cv2.dilate(opening, kernel, iterations=2) #adds pixels to the boundaries of objects in an image
        #eroded = cv2.erode(dilation, kernel, iterations=1 ) #removes pixels on object boundaries        
        '''
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        minarea = 500
        # max area for contours, can be quite large for buses
        maxarea = 35000
        # vectors for the x and y locations of contour centroids in current frame
        '''cxx = np.zeros(len(contours))
        cyy = np.zeros(len(contours))
        '''
        temp = 0
        for i in range(len(contours)):  # cycles through all contours in current frame
            if hierarchy[0, i, 3] == -1:  # using hierarchy to only count parent contours (contours not within others)
                area = cv2.contourArea(contours[i])  # area of contour
                if minarea < area < maxarea:  # area threshold for contour
                    # calculating centroids of contours
                    cnt = contours[i]
                    M = cv2.moments(cnt)
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                    
                    x, y, w, h = cv2.boundingRect(cnt)
                    
                    xA = x
                    xB = x + w
                    yA = y
                    yB = y + h
                    if w  > 320 or w < 50 or h < 22 or h > 90:
                        continue
                    if w > 320 and h > 90:
                        continue
                        
                    not_matched = True
                    noteIdx = -1
                    for idx, centroid_data in enumerate(centroids_list):
                        if centroid_data[0] == count:
                            continue
            
                        # check proximity using manhattan distance
                        X = (float(xA + xB) / 2 - float(centroid_data[2][0] + centroid_data[2][2]) / 2)
                        Y = (float(yA + yB) / 2 - float(centroid_data[2][1] + centroid_data[2][3]) / 2)

                        if abs(X) < 70 and abs(Y) < 15:  # 50,15(e)
                            noteIdx = idx
                            
                            if h < 40 and abs(X) < 60 and abs(Y) < 10: #40,35
                                #print("Bike ID:", centroid_data[7], ":",centroid_data[2], [xA,yA], X, Y)
                                not_matched = False
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6][0].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                if len(centroids_list[idx][6][0]) > 400:
                                    centroids_list[idx][6][0].pop(0)
                                    
                                if centroids_list[idx][5] == "unlocked":
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                                        if X > 0:
                                            centroids_list[idx][4][2] += 1  # To track towards right movements
                                        else:
                                            centroids_list[idx][4][3] += 1 # To track towards left movements
                                    else:
                                        centroids_list[idx][3] = 0
                
                                if centroids_list[idx][3] == 6:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                    if centroids_list[idx][4][2] > centroids_list[idx][4][3]:
                                        centroids_list[idx][4] = ["Bike(R)", "right", 0, 0]
                                    else:
                                        centroids_list[idx][4] = ["Bike(L)", "left", 0, 0]
                                        
                                if centroids_list[idx][6][-1] != 0.0:
                                    centroids_list[idx][6][1].append(int((sum(centroids_list[idx][6][0])/len(centroids_list[idx][6][0]))))
                                    if len(centroids_list[idx][6][1]) > 10:
                                        centroids_list[idx][6][1].pop(0)
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)  # (255,20,147) = pink color
                                    cv2.putText(image, str(int(centroids_list[idx][6][1][-1]))+ str("KMPH")  + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 2, cv2.LINE_AA) 
                                    cv2.putText(image, str(centroids_list[idx][4][0]),(centroid_data[2][0]+90, centroid_data[2][1]+25) ,
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,20,147), 2, cv2.LINE_AA)
                                    
                            elif h > 70 or w > 180:   #40, 185
                                #print("Bus:",h,w)
                                not_matched = False
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6][0].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                if len(centroids_list[idx][6][0]) > 400:
                                    centroids_list[idx][6][0].pop(0)
                                    
                                if centroids_list[idx][5] == "unlocked":
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                                        if X > 0:
                                            centroids_list[idx][4][2] += 1  # To track towards right movements
                                        else:
                                            centroids_list[idx][4][3] += 1 # To track towards left movements
                                    else:
                                        centroids_list[idx][3] = 0
                                else:
                                    if X > 0:
                                        centroids_list[idx][4][2] = ["Bus(R)", "right", 0, 0]
                                    else:
                                        centroids_list[idx][4][3] = ["Bus(R)", "right", 0, 0]
                
                                if centroids_list[idx][3] == 15:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                    if centroids_list[idx][4][2] > centroids_list[idx][4][3]:
                                        centroids_list[idx][4] = ["Bus(R)", "right", 0, 0]
                                    else:
                                        centroids_list[idx][4] = ["Bus(L)", "left", 0, 0]
                                        
                                if centroids_list[idx][6][-1] != 0.0:
                                    centroids_list[idx][6][1].append(int((sum(centroids_list[idx][6][0])/len(centroids_list[idx][6][0]))))
                                    if len(centroids_list[idx][6][1]) > 10:
                                        centroids_list[idx][6][1].pop(0)
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                    cv2.putText(image, str(int(centroids_list[idx][6][1][-1]))+ str("KMPH")  + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 2, cv2.LINE_AA) 
                                    cv2.putText(image, str(centroids_list[idx][4][0]),(centroid_data[2][0], centroid_data[2][1]+40) ,
                                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,20,147), 2, cv2.LINE_AA)
                            else:          
                                #print("Car ID:", centroid_data[7], ":",centroid_data[2], [xA,yA], X, Y, h,w)
                                not_matched = False
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6][0].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                if len(centroids_list[idx][6][0]) > 400:
                                    centroids_list[idx][6][0].pop(0)
                                    
                                if centroids_list[idx][5] == "unlocked":
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                                        if X > 0:
                                            centroids_list[idx][4][2] += 1  # To track towards right movements
                                        else:
                                            centroids_list[idx][4][3] += 1 # To track towards left movements
                                    else:
                                        centroids_list[idx][3] = 0
                
                                if centroids_list[idx][3] == 16:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                    if centroids_list[idx][4][2] > centroids_list[idx][4][3]:
                                        centroids_list[idx][4] = ["Car(R)", "right", 0, 0]
                                    else:
                                        centroids_list[idx][4] = ["Car(L)", "left", 0, 0]
                                        
                                if centroids_list[idx][6][-1] != 0.0:
                                    centroids_list[idx][6][1].append(int((sum(centroids_list[idx][6][0])/len(centroids_list[idx][6][0]))))
                                    if len(centroids_list[idx][6][1]) > 10:
                                        centroids_list[idx][6][1].pop(0)
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                    cv2.putText(image, str(int(centroids_list[idx][6][1][-1]))+ str("KMPH")  + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 2, cv2.LINE_AA) 
                                    cv2.putText(image, str(centroids_list[idx][4][0]),(centroid_data[2][0], centroid_data[2][1]+40) ,
                                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,20,147), 2, cv2.LINE_AA)
                                
                            centroids_list[idx][0] = count
                            
                            break
            
                    if not_matched:
                        if count < 125:
                            color = make_new_color()
                            # append new rectangle in previous cars list
                            #car_count = random.randint(50, 200) 
                            centroids_list.appendleft([count, color, (xA, yA, xB, yB), 1, ["NA", "NA", 0, 0], "unlocked", [[0], [0]], car_count])
                            car_count += 1
                            print("Vehicle No.:", centroids_list[0][7],"arrived")
                            new_ID = True
                        elif xA < 600 or xA > 3500:
                            color = make_new_color()
                            # append new rectangle in previous cars list
                            #car_count = random.randint(50, 200) 
                            centroids_list.appendleft([count, color, (xA, yA, xB, yB), 1, ["NA", "NA", 0, 0], "unlocked", [[0], [0]], car_count])
                            car_count += 1
                            print("Vehicle No.:", centroids_list[0][7],"arrived")
                            new_ID = True
                        
                    #print("centroids:",centroids_list[noteIdx][7],centroids_list[noteIdx][5],centroids_list[noteIdx][4][1],noteIdx)
                    # Computing lateral distances of nearby locked vehicles
                    for idx, centroid_data in enumerate(centroids_list):
                        if  noteIdx != -1 and centroids_list[noteIdx][5] == "locked" and centroids_list[noteIdx][4][1] == "right":
                            if abs(centroids_list[idx][2][0] - centroids_list[noteIdx][2][0]) < 100 :
                                if centroids_list[idx][4][1] == "left":
                                    lateral_dist = abs(centroids_list[noteIdx][2][3] - centroids_list[idx][2][1])
                                    accelearation = (abs(centroids_list[noteIdx][6][1][-1]-centroids_list[noteIdx][6][1][-2]))//60
                                    #accelearationCar = (abs(speedCar[-1]-speedCar[-2]))//60
                                    #accelearationBus = (abs(speedBus[-1]-speedBus[-2]))//60
                                    #print("acceleration=",accelearation)
                                    #print("ID =",centroids_list[noteIdx][7],centroids_list[idx][7],"Lat Dist=", lateral_dist)
                                    (x1, y1) = (int((centroids_list[noteIdx][2][0] + centroids_list[noteIdx][2][2])/2),centroids_list[noteIdx][2][3])
                                    (x2, y2) = (int((centroids_list[idx][2][0] + centroids_list[idx][2][2])/2),centroids_list[idx][2][1])
                                    #cv2.line(image, (x1, y1), (x2, y2), [0,0,255], 4)
                                    #Writing into a csv file
                                    with open("lateral_distance.csv", "a") as o:
                                        o.write(str("lateral distance between IDs: ") + str(centroids_list[noteIdx][7]) + str(" and ") + str(centroids_list[idx][7]) + " = " + str(lateral_dist) + "\n")
                                    cv2.putText(image, str("Lateral distance between IDs: ")+ str(centroids_list[noteIdx][7]) + str(" and ") + str(centroids_list[idx][7]) + " = " + str(lateral_dist),
                                                (2350, image.shape[0] - 495 + temp), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 200), 3)
                                    temp += 50
                        
                    if centroids_list[0][5] == "locked":
                        for veh_ID in range(len(centroids_list)):
                            temp_new_ID += 70
                            cv2.putText(image, str(centroids_list[veh_ID][4][0]) + str("  with ") + str("ID-") +  str(centroids_list[veh_ID][7]) + str("  has arrived"),   
                                    (50, image.shape[0] - 520 + temp_new_ID),cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 200, 200), 3)
                        if veh_ID == len(centroids_list) -1:
                            temp_new_ID = 0
                    centroids_list = deque([car_data for car_data in list(centroids_list) if count - car_data[0] < 100])

                            
    image[500:955 ,] = image2[500:955, ] ;image[1200:1600 ,] = image2[1200:1600 ,]  # 970(e), 955(n)   # For single video
    #image[150:651 ,] = image2[150:651, ] ;image[900:1300 ,] = image2[900:1300 ,]  # 970(e), 955(n)  # FOR STICHED VIDEOS
    image[(image.shape[0] - 110):image.shape[0] , int(image.shape[1]*0.62):] = 255
    timestamp = datetime.datetime.now()
    ts = timestamp.strftime(" %d %B %Y %I:%M:%S%p")

    cv2.putText(image, str("R-") + str("means") +  str(" going towards right"), (1250, image.shape[0] - 140),  
        cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 150, 200), 3)
    cv2.putText(image, str("L-") + str("means") +  str(" going towards left"), (1250, image.shape[0] - 40),  
        cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 150, 200), 3)
    cv2.putText(image, "Drone (Bird's Eye View)", (1400, 150),  
        cv2.FONT_HERSHEY_SIMPLEX, 4.0, (255, 255, 0), 3)
    cv2.putText(image, "[Traffic Vehicles Detection, Tracking and Speed Estimation]", (600, 300),  
        cv2.FONT_HERSHEY_SIMPLEX, 3.0, (255, 0, 200), 3)
    cv2.putText(image, ts, (2500, image.shape[0] - 20),  
        cv2.FONT_HERSHEY_SIMPLEX, 3.0, (0, 0, 200), 3)  # TimeStamp
    cv2.putText(image, "Drone-1", (1400,350),
        cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 0), 4) # Drone-1
    cv2.putText(image, "Drone-2", (4000, 350),
        cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 0), 4) # Drone-2
    #cv2.imwrite("img.png",image)
    image = im.resize(image, 2500)
    writer.write(image)
    #cv2.imshow('output', image)
    # Wait for Esc key to stop
    if cv2.waitKey(33) == 27:
        break 
    # outputs all the video frames into out folder present in the working directory
    #cv2.imwrite("/content/drive/My Drive/Colab Notebooks/out/" + str(count) + ".jpg", image)
    count += 1
    #print("Frame Count=",count)

print("Number of vehicle crossed = ", car_count)
print("Number of frames processed = ", frame_count)
Time_Elapsed = time.time()-start
print("Time Elapsed=",Time_Elapsed,'s')
print("finished Successfully")
print(image.shape)
cap.release()
writer.release()
cv2.destroyAllWindows()
# Output: https://drive.google.com/drive/folders/12HLHqqccQvQoCJFofr83Kava4-CcVcK3 


INFO: Video Starting
Video Open- True
No. of frames: 5870
INFO: Tracking is in Progress...
Vehicle No.: 0 arrived
Vehicle No.: 1 arrived
Vehicle No.: 2 arrived
Vehicle No.: 3 arrived
Vehicle No.: 4 arrived
Vehicle No.: 5 arrived
Vehicle No.: 6 arrived
Vehicle No.: 7 arrived
Vehicle No.: 8 arrived
Vehicle No.: 9 arrived
Vehicle No.: 10 arrived
Vehicle No.: 11 arrived
Vehicle No.: 12 arrived
Vehicle No.: 13 arrived
Vehicle No.: 14 arrived
Vehicle No.: 15 arrived
Vehicle No.: 16 arrived
Number of vehicle crossed =  17
Number of frames processed =  3600
Time Elapsed= 309.75139236450195 s
finished Successfully
(1367, 2500, 3)


In [ ]:
# Locations: /content/drive/My Drive/Colab Notebooks/
# -*- coding: utf-8 -*-
"""
Created on Mon June 1 22:47:29 2020
Updated on Mon June 12
@author: Kumar Gaurav
Institutions: EE, IIT Kanpur

Title : Traffic Vehicle Speed Estimation
------------------------------------------------------------------
calculates speed of vehicles in realtime in KMPH
this code mainly focusses on detecting and tracking cars by using trained  classifier which already has features of cars

key_formula =  ( speed in object plane / distance of camera from road ) = ( speed in image plane / focal length )
approx fps of the video = 30

to obtain accurate value distance to object (mm)  =   focal length (mm) * real height of the object (mm) * image height (pixels)
                                                       ----------------------------------------------------------------
                                                                 object height (pixels) * sensor height (mm)
Default Code
Updated 29 Jun 20
"""
import cv2
from google.colab.patches import cv2_imshow # Colab cv2 imshow
import numpy as np
import random
from collections import deque
import imutils as im
import time


def make_new_color():
    return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
def make_new_veh():
    return [random.randint(1, 50)]
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    result = result[:,50:4000]
    result1 = result.copy()
    result[:985 ,] = 0 ;result[1170: ,] = 0
    return result, result1
def shadow_removal(img):
    rgb_planes = cv2.split(img)
    
    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((5,5), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)
    
    result = cv2.merge(result_planes)
    result_norm = cv2.merge(result_norm_planes)
    return result_norm

# Sensor Details:
GSD = 0.0366 #3.66 cm/pixels
Iw = 4096; Sw = 13.2 # in mm
value = (GSD*Iw)/Sw
f = 8.8; distance = 100 # approximated, real distance = (R/r)*(f*W/w)
ratio = distance/f
count = 0

#/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4
# capt.Video frames from a video
print("INFO: Video Starting")
path = '/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4'
cap = cv2.VideoCapture(path)

print("Video Open-",cap.isOpened())
# Subtractors
sub = cv2.createBackgroundSubtractorMOG2()  # create background subtractor
mogSubtractor = cv2.bgsegm.createBackgroundSubtractorMOG(300)
mog2Subtractor = cv2.createBackgroundSubtractorMOG2() 
gmgSubtractor = cv2.bgsegm.createBackgroundSubtractorGMG(10, .8)
knnSubtractor = cv2.createBackgroundSubtractorKNN()   #Very efficient if number of foreground pixels is low
cntSubtractor = cv2.bgsegm.createBackgroundSubtractorCNT(5, True)
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,12))

centroids_list = deque([])
car_count = 0
'''with open("/content/drive/My Drive/Colab Notebooks/out1.csv", "w") as o:
    pass'''

#width = 1500 ; height = 820
width = 2000; height = 1093
#frate = int(cap.get(cv2.CAP_PROP_FPS))
frate = 10
#***************************************************************************************************************
path21 = '/content/drive/My Drive/Colab Notebooks/Videos/_Track_KNN_final20.mp4' # Video write
writer = cv2.VideoWriter(path21, cv2.VideoWriter_fourcc(*'XVID'),frate, (width, height))

print("INFO: Tracking is in Progress...")
frame_count = 0
start = time.time()

while True:

    ret, frame = cap.read()  # import image
    frame_count +=  1
    if frame_count == 5:  #3627
      break

    if  ret:  # if there is a frame continue with code
        image, image2 = rotate_image(frame, 1.78)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # converts image to gray
        image1 = shadow_removal(gray)
        fg_mask = knnSubtractor.apply(image1)  #  mog2Subtractor  knnSubtractor
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        value, th = cv2.threshold(fg_mask, 10, 255, cv2.THRESH_BINARY)
        opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=1) #erosion followed by dilation, useful in removing noises
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=3) #Dilation followed by Erosion
        dilation = cv2.dilate(closing, kernel, iterations = 1)
        '''        
        dilation = cv2.dilate(opening, kernel, iterations=2) #adds pixels to the boundaries of objects in an image
        #eroded = cv2.erode(dilation, kernel, iterations=1 ) #removes pixels on object boundaries        
        '''
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        minarea = 700
        # max area for contours, can be quite large for buses
        maxarea = 18000
        # vectors for the x and y locations of contour centroids in current frame
        '''cxx = np.zeros(len(contours))
        cyy = np.zeros(len(contours))
        '''
        for i in range(len(contours)):  # cycles through all contours in current frame
            if hierarchy[0, i, 3] == -1:  # using hierarchy to only count parent contours (contours not within others)
                area = cv2.contourArea(contours[i])  # area of contour
                if minarea < area < maxarea:  # area threshold for contour
                    # calculating centroids of contours
                    cnt = contours[i]
                    M = cv2.moments(cnt)
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                    
                    x, y, w, h = cv2.boundingRect(cnt)
                    
                    xA = x
                    xB = x + w
                    yA = y
                    yB = y + h
                    if w  > 300 or w < 50:
                        continue
                    elif h < 20 or h > 90:
                        continue
                    elif w > 300 and h > 90:
                        continue
                        
                    not_matched = True
                    for idx, centroid_data in enumerate(centroids_list):
                        if centroid_data[0] == count:
                            continue
                        if centroids_list[idx][4] == 0:
                            centroids_list[idx][5] = "unlocked"
                            centroids_list[idx][4] = 5
            
                        # check proximity using manhattan distance
                        X = abs(float(centroid_data[2][0] + centroid_data[2][2]) / 2 - float(xA + xB) / 2)
                        Y = abs(float(centroid_data[2][1] + centroid_data[2][3]) / 2 - float(yA + yB) / 2)

                        if X < 35 and Y < 3:
            
                            not_matched = False
                            centroids_list[idx][4] = 5
                            centroids_list[idx][2] = [xA, yA, xB, yB]
                            #*********************************************************************************************************
                            #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                            centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                            if centroids_list[idx][5] == "unlocked":
            
                                if centroids_list[idx][0] == count - 1:
                                    centroids_list[idx][3] += 1
            
                                else:
                                    centroids_list[idx][3] = 0
            
                            if centroids_list[idx][3] == 3:
                                centroids_list[idx][5] = "locked"
                                centroids_list[idx][3] = 0
                            if centroids_list[idx][6][-1] != 0.0:
                                cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                cv2.putText(image, str(int(centroids_list[idx][6][-1]))+ str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                            (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 0, cv2.LINE_AA) 
                                if h < 40:
                                    cv2.putText(image, str("bike"),
                                                (centroid_data[2][0], centroid_data[2][1]+40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,20,147), 1, cv2.LINE_AA)
                                elif h > 40 and w > 200:
                                    cv2.putText(image, str("Bus"),
                                                (centroid_data[2][0], centroid_data[2][1]+40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,20,147), 2, cv2.LINE_AA)
                                else:
                                    cv2.putText(image, str("Car"),
                                                (centroid_data[2][0], centroid_data[2][1]+40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,20,147), 2, cv2.LINE_AA)
                                
                            centroids_list[idx][0] = count
                            break
            
                    if not_matched:
                        color = make_new_color()
            
                        # append new rectangle in previous cars list
                        car_count = random.randint(1, 40) 
                        centroids_list.appendleft([count, color, (xA, yA, xB, yB), 1, 5, "unlocked", [0], car_count])
                        #car_count += 1
                        # cv2.rectangle(image, (xA, yA), (xB, yB), color, 2)
                        # cv2.putText(image, "0",
                        #             (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
                        prev_color = color
                        prev_coords = [xA, yA, xB, yB]
            
                    # plot all remaining locked rectangles
                    for idx, centroid_data in enumerate(centroids_list):
                
                        if centroid_data[5] == "locked" and centroid_data[0] != count:
                            centroids_list[idx][4] -= 1
                            if centroids_list[idx][6][-1] != 0.0:
                                cv2.rectangle(image, (centroid_data[2][0], centroid_data[2][1]), (centroid_data[2][2], centroid_data[2][3]),
                                              centroid_data[1], 2)

                                cv2.putText(image, str(int(centroids_list[idx][6][-1])) + str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                            (centroid_data[2][0], centroid_data[2][1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 0,
                                            cv2.LINE_AA) 
                                
                            if centroids_list[idx][4] == 0:
                              centroids_list[idx][5] = "unlocked"
                              centroids_list[idx][4] = 5
                              centroids_list[idx][3] = 0
                                
                        if count - centroid_data[0] == 10:
                            if sum(centroid_data[6]) / len(centroid_data[6]) != 0.0:
                                with open("out1.csv", "a") as o:
                                    o.write(str(centroid_data[7]) + ": " + str(sum(centroid_data[6]) / len(centroid_data[6])) + "\n")
                
                    centroids_list = deque([car_data for car_data in list(centroids_list) if count - car_data[0] < 10])
                    
#                else:
#                    break
     
    image[500:985 ,] =image2[500:985, ] ;image[1170:1600 ,] = image2[1170:1600 ,]

    image = im.resize(image, 2000)
    writer.write(image)
    '''cv2.imshow('output', closing)
    # Wait for Esc key to stop
    if cv2.waitKey(33) == 27:
        break '''
    # outputs all the video frames into out folder present in the working directory
    #cv2.imwrite("/content/drive/My Drive/Colab Notebooks/out/" + str(count) + ".jpg", image)
    count += 1

Time_Elapsed = time.time()-start
print("Time Elapsed=",Time_Elapsed,'s')
print("finished Successfully")
print(image.shape)
cap.release()
writer.release()
cv2.destroyAllWindows()
# Output: https://drive.google.com/drive/folders/12HLHqqccQvQoCJFofr83Kava4-CcVcK3 


INFO: Video Starting
Video Open- True
INFO: Tracking is in Progress...
Time Elapsed= 2244.2843549251556 s
finished Successfully
(1093, 2000, 3)


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 15 16:28:37 2020
updated on July 15
Re- updated on Aug 27
@author: KGaurav
Title : Align Drone Video Frames
Directory: Work
"""

import cv2
import numpy as np
import os
import sys
sys.path.insert(0, os.path.abspath("/content/drive/My Drive/Colab Notebooks/"))
from align import alignImages

# Path of dataset directory 
#path = "E:\CodeVideoData\_DroneData\ClipDroneData\_1stDrone.MP4"  # _1stDrone_2~2
print("INFO: Video Starting")
#path = '/content/drive/My Drive/Colab Notebooks/Videos/3rdDrone.MP4'  
#path = '/content/drive/My Drive/Colab Notebooks/Colab Notebooks/Videos/DJI_Morning.MP4' # 1stDrone DJI_Morning
path = '/content/drive/My Drive/Colab Notebooks/Colab Notebooks Anshu/Videos/_2ndDrone_align.mp4'  # 1stDrone_newAlign_01
cap = cv2.VideoCapture(path)  
print("Video Status: ",cap.isOpened())

#cv2.namedWindow("output", cv2.WINDOW_NORMAL)
frame_count = 0

frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT ))
#width = 1500; height = 820
width = 4096; height = 2160
#width = 4096; height = 432
frate = int(cap.get(cv2.CAP_PROP_FPS))
#frate = 10
#***************************************************************************************************************
path21 = '/content/drive/My Drive/Colab Notebooks/Videos/_2ndDrone_NewAlign_01.mp4' # Video write
writer = cv2.VideoWriter(path21, cv2.VideoWriter_fourcc(*'XVID'),frate, (width, height))
first_frame = True
print("INFO: Aligning is in Progress...",frames)

while(cap.isOpened()):
    ret, frame = cap.read()
    frame_count += 1
    if frame_count == 3620: # 10800,  3663
      break
    if first_frame:
        imgRef = frame
        first_frame = False
    image = alignImages(frame , imgRef)
    #result[0:864 ,] = frame[0:864, ] ;result[1296: ,] = frame[1296: ,]
    #print("frame count = ",frame_count)
    #cv2.imwrite("out/" + str(frame_count) + ".jpg",alignImage)
    #cv2.imwrite("houghlines_result.jpg",frame)
    writer.write(image)

  
# destroy all the windows that is currently on 
print("Frame Size=",image.shape)
cap.release() 
writer.release()
cv2.destroyAllWindows()  



INFO: Video Starting
Video Status:  True
INFO: Aligning is in Progress... 3659
Frame Size= (2160, 4096, 3)


In [ ]:
"""                                             
Working CODE 1  : absdiff                                                             
Usage: 
Updated!
"""
# Motion Detection realtime
import cv2
from google.colab.patches import cv2_imshow # Colab cv2 imshow
import numpy as np
import imutils
import datetime
import time
import random
from collections import deque
import imutils as im

def main():
    
    w = 800
    h = 600
    def make_new_color():
        return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]

    def rotate_image(image, angle):
        image_center = tuple(np.array(image.shape[1::-1]) / 2)
        rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
        result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
        result = result[:,50:4000]
        result1 = result.copy()
        result[:985 ,] = 0 ;result[1170: ,] = 0
        return result, result1

    # Sensor Details:
    GSD = 0.0366 #3.66 cm/pixels
    Iw = 4096; Sw = 13.2 # in mm
    value = (GSD*Iw)/Sw
    f = 8.8; distance = 100 # approximated, real distance = (R/r)*(f*W/w)
    ratio = distance/f
    count = 0
       
    path = '/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4'
    cap = cv2.VideoCapture(path)
    width = 2936
    height = 594
    #frate = int(cap.get(cv2.CAP_PROP_FPS))

    cap.set(3, w)
    cap.set(4, h)
    
#    print(cap.get(3))
#    print(cap.get(4))
    
    if cap.isOpened():
        ret, frame = cap.read()
    else:
        ret = False
    

    frame_count = 0
    print("[INFO] Vehicle detection is in progress...")
    centroids_list = deque([])
    car_count = 0
    with open("/content/drive/My Drive/Colab Notebooks/out2.csv", "w") as o:
      pass

    #width = 4096, height = 2160
    width = 1500; height = 820
    #frate = int(cap.get(cv2.CAP_PROP_FPS))
    frate = 10
    #***************************************************************************************************************
    path21 = '/content/drive/My Drive/Colab Notebooks/Videos/_Track_absdiff_06.mp4' # Video write
    writer = cv2.VideoWriter(path21, cv2.VideoWriter_fourcc(*'XVID'),frate, (width, height))

    start_frame = True
    count_frame = 0
    print("Start...")
    while ret:
        if start_frame:
            ret, frame1 = cap.read()
            ret, frame2 = cap.read()
            frame1, img = rotate_image(frame1, 1.78)
            frame2, img2 = rotate_image(frame2, 1.78)
            start_frame = False
        else:
            frame2, img = rotate_image(frame2, 1.78)
            
        count_frame += 1
        if count_frame == 2000:  #3500
          break
        diff = cv2.absdiff(frame1, frame2)
        grey = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (3, 3), 0)
        #retvalbin, bins = cv2.threshold(blur, 10, 255, cv2.THRESH_BINARY)  #
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 3))  # kernel to apply to the morphology
        closing = cv2.morphologyEx(blur, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations=1)
        
        dilation = cv2.dilate(opening, kernel, iterations=2) #adds pixels to the boundaries of objects in an image
        closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel, iterations=1)
        #eroded = cv2.erode(dilation, kernel, iterations=1 ) #removes pixels on object boundaries
        #dilation = cv2.dilate(dilation, kernel)
        
        retvalbin, bins = cv2.threshold(dilation, 20, 255, cv2.THRESH_BINARY)  # removes the shadows
        
        # creates contours
        # cv2.imshow('bins',bins)
        contours, hierarchy = cv2.findContours(bins, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        minarea = 400
        # max area for contours, can be quite large for buses
        maxarea = 50000
        # vectors for the x and y locations of contour centroids in current frame
        cxx = np.zeros(len(contours))
        cyy = np.zeros(len(contours))
        
        for i in range(len(contours)):  # cycles through all contours in current frame
            if hierarchy[0, i, 3] == -1:  # using hierarchy to only count parent contours (contours not within others)
                area = cv2.contourArea(contours[i])  # area of contour
                if minarea < area < maxarea:  # area threshold for contour
                    # calculating centroids of contours
                    cnt = contours[i]
                    M = cv2.moments(cnt)
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                    
                    # gets bounding points of contour to create rectangle
                    # x,y is top left corner and w,h is width and height
                    x, y, w, h = cv2.boundingRect(cnt)
                    
                    xA = x
                    xB = x + w
                    yA = y
                    yB = y + h

                    if w  > 300 or w < 50:
                        continue
                    elif h < 20 or h > 90:
                        continue
                    elif w > 300 and h > 90:
                        continue

                    # Enumerate over all the cars in centroids_list
                    #each centroid_list element contains: [last_updated_frame, color, position, 
                    #lock_count, unlock_count, lockstate(unlocked by default), list_of_car_speeds_in_prev_frames, id] 
                    not_matched = True
                    for idx, centroid_data in enumerate(centroids_list):
                        if centroid_data[0] == count:
                            continue
                        if centroids_list[idx][4] == 0:
                            centroids_list[idx][5] = "unlocked"
                            centroids_list[idx][4] = 5
            
                        # check proximity using manhattan distance
                        X = abs(float(centroid_data[2][0] + centroid_data[2][2]) / 2 - float(xA + xB) / 2)
                        Y = abs(float(centroid_data[2][1] + centroid_data[2][3]) / 2 - float(yA + yB) / 2)
                        # if there is a rectangle in 10 pixel proximity of a rectangle of previous frame than i am assuming that,
                        # the car in the rectangle is same as it was in the previous frame
                        # 10 can be changed to any other value based on the movement happening in the frames, if vehicles are moving
                        # more than 10 pixels per frame suppose 20 so change the value to 20
                        if X < 35 and Y < 2:
            
                            not_matched = False
                            centroids_list[idx][4] = 5
                            centroids_list[idx][2] = [xA, yA, xB, yB]
                            #*********************************************************************************************************
                            #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                            centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)                            
                            if centroids_list[idx][5] == "unlocked":
            
                                if centroids_list[idx][0] == count - 1:
                                    centroids_list[idx][3] += 1
            
                                else:
                                    centroids_list[idx][3] = 0
            
                            if centroids_list[idx][3] == 3:
                                centroids_list[idx][5] = "locked"
                                centroids_list[idx][3] = 0
                            if centroids_list[idx][6][-1] != 0.0:
                                cv2.rectangle(frame1, (xA, yA), (xB, yB), centroid_data[1], 2)
                                cv2.putText(frame1, str(int(centroids_list[idx][6][-1]))+ str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                            (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                
                            centroids_list[idx][0] = count
                            break
            
            #    If rectangle not matches with previous rectangles that means it is a new car so make a new rectangle
                    #if rectangle is not matching wiht previous rectangles ,then it is assumed that a new car has come and so new rectangle
                    if not_matched:
                        color = make_new_color()
            
                        # append new rectangle in previous cars list
                        car_count = random.randint(1, 50)
                        centroids_list.appendleft([count, color, (xA, yA, xB, yB), 1, 5, "unlocked", [0], car_count])
                        #car_count += 1
                        # cv2.rectangle(image, (xA, yA), (xB, yB), color, 2)
                        # cv2.putText(image, "0",
                        #             (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
                        prev_color = color
                        prev_coords = [xA, yA, xB, yB]
            
                # plot all remaining locked rectangles
                for idx, centroid_data in enumerate(centroids_list):
            
                    if centroid_data[5] == "locked" and centroid_data[0] != count:
                        centroids_list[idx][4] -= 1
                        if centroids_list[idx][6][-1] != 0.0:
                            cv2.rectangle(frame1, (centroid_data[2][0], centroid_data[2][1]), (centroid_data[2][2], centroid_data[2][3]),
                                          centroid_data[1], 2)
                           # cv2.putText(image, str(centroids_list[idx][6][-1]),
                            #            (centroid_data[2][0], centroid_data[2][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, centroid_data[1], 1,
                             #           cv2.LINE_AA)
                            cv2.putText(frame1, str(int(centroids_list[idx][6][-1])) + str("KMPH")  + str(",ID=") + str(centroids_list[idx][7]),
                                        (centroid_data[2][0], centroid_data[2][1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1,
                                        cv2.LINE_AA)  
                            
                        if centroids_list[idx][4] == 0:
                            centroids_list[idx][5] = "unlocked"
                            centroids_list[idx][4] = 5
                            centroids_list[idx][3] = 0
                            
                    if count - centroid_data[0] == 10:
                        if sum(centroid_data[6]) / len(centroid_data[6]) != 0.0:
                            with open("out1.csv", "a") as o:
                                o.write(str(centroid_data[7]) + ": " + str(sum(centroid_data[6]) / len(centroid_data[6])) + "\n")
            
                centroids_list = deque([car_data for car_data in list(centroids_list) if count - car_data[0] < 10])
                

        frame1[500:985 ,] = img[500:985, ] ;frame1[1170:1600 ,] = img[1170:1600 ,]
        frame1 = im.resize(frame1, 1500)
        #print(frame1.shape)
        
        writer.write(frame1)
        '''cv2.imshow('Output', frame1)
        # Wait for Esc key to stop
        if cv2.waitKey(33) == 27:
            break '''
        # outputs all the video frames into out folder present in the working directory
        cv2.imwrite("E:/CodeVideoData/NewStitch2/out/" + str(count) + ".jpg", frame1)
        count += 1
        frame1 = frame2
        ret, frame2 = cap.read()
    

    print("finished Successfully")
    
    cap.release()
    writer.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    print("Start")
    start = time.time()
    main()
    print("Finished")
    Time_Elapsed = time.time()-start
    print("Time Elapsed=",Time_Elapsed,'s')
    

Start
[INFO] Vehicle detection is in progress...
Start...
finished Successfully
Finished
Time Elapsed= 500.95209884643555 s


In [ ]:
# Locations: /content/drive/My Drive/Colab Notebooks/
# -*- coding: utf-8 -*-
"""
Created on Mon June 1 22:47:29 2020
Updated on Mon July 15
@author: Kumar Gaurav
Institutions: EE, IIT Kanpur

Title : Traffic Vehicle Speed Estimation
------------------------------------------------------------------
calculates speed of vehicles in realtime in KMPH

key_formula =  ( speed in object plane / distance of camera from road ) = ( speed in image plane / focal length )
approx fps of the video = 30

to obtain accurate value distance to object (mm)  =   focal length (mm) * real height of the object (mm) * image height (pixels)
                                                       ----------------------------------------------------------------
                                                                 object height (pixels) * sensor height (mm)
Default Code
Updated 29 Jun 20
"""
import cv2
from google.colab.patches import cv2_imshow # Colab cv2 imshow
import numpy as np
import random
from collections import deque
import imutils as im
import time
import sys
sys.path.insert(0, "/content/drive/My Drive/Colab Notebooks/")
from align import alignImages

def make_new_color():
    return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
def make_new_veh():
    return [random.randint(1, 200)]
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    result = result[:,50:4000]
    result1 = result.copy()
    #result[:985 ,] = 0 ;result[1170: ,] = 0
    result[:970 ,] = 0 ;result[1200: ,] = 0
    return result, result1

def shadow_removal(img):
    rgb_planes = cv2.split(img)
    
    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((5,5), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)
    
    result = cv2.merge(result_planes)
    result_norm = cv2.merge(result_norm_planes)
    return result_norm
def canny_edge_detector(image): 
      
    # Convert the image color to grayscale 
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)  
      
    # Reduce noise from the image 
    blur = cv2.GaussianBlur(gray_image, (5, 5), 0)  
    canny = cv2.Canny(blur, 50, 150) 
    return canny 
def region_of_interest(image): 
    #height = image.shape[0] 
    '''polygons = np.array([ 
        [(200, height), (1100, height), (550, 250)]   # (width, height)
        ]) '''
    #polygons = np.array([[ (0, 900), (0,1250), (image.shape[1],1250), (image.shape[1], 900) ]])
    #polygons = np.array([[ (0, 1030), (0,1150), (image.shape[1],1150), (image.shape[1], 1030) ]])
    polygons = np.array([[ (0, 1100), (0,1230), (image.shape[1],1230), (image.shape[1], 1100) ]])
    mask = np.zeros_like(image) 
    # Fill poly-function deals with multiple polygon 
    cv2.fillPoly(mask, polygons, 255)  
      
    # Bitwise operation between canny image and mask image 
    masked_image = cv2.bitwise_and(image, mask)  
    return masked_image
def getTransform(image, ty):
    rows,cols = image.shape[:2]
    #pts1 = np.float32([[50,50],[200,50],[50,200]])
    #pts2 = np.float32([[10,100],[200,50],[100,250]])
    #M = cv2.getAffineTransform(pts1,pts2)
    ty = ty - cols
    M = np.float32([[1,0,0],[0,1,ty]])
    dst = cv2.warpAffine(image,M,(cols,rows))
    return dst

# Sensor Details:
GSD = 0.0366 #3.66 cm/pixels
Iw = 4096; Sw = 13.2 # in mm
value = (GSD*Iw)/Sw
f = 8.8; distance = 100 # approximated, real distance = (R/r)*(f*W/w)
ratio = distance/f
count = 0

#/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4
# capt.Video frames from a video
print("INFO: Video Starting")
path = '/content/drive/My Drive/Colab Notebooks/Videos/DJI_Morning.MP4'  # 1stDrone DJI_Morning
cap = cv2.VideoCapture(path)

print("Video Open-",cap.isOpened())
# Subtractors
sub = cv2.createBackgroundSubtractorMOG2()  # create background subtractor
mogSubtractor = cv2.bgsegm.createBackgroundSubtractorMOG(300)
mog2Subtractor = cv2.createBackgroundSubtractorMOG2() 
gmgSubtractor = cv2.bgsegm.createBackgroundSubtractorGMG(10, .8)
knnSubtractor = cv2.createBackgroundSubtractorKNN()   #Very efficient if number of foreground pixels is low
cntSubtractor = cv2.bgsegm.createBackgroundSubtractorCNT(5, True)
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,12))

centroids_list = deque([])
car_count = 0
'''with open("/content/drive/My Drive/Colab Notebooks/out1.csv", "w") as o:
    pass'''

#width = 1500 ; height = 820
width = 2000; height = 1093
#frate = int(cap.get(cv2.CAP_PROP_FPS))
frate = 20
#***************************************************************************************************************
path21 = '/content/drive/My Drive/Colab Notebooks/Videos/_Track_KNN_align_04.mp4' # Video write
writer = cv2.VideoWriter(path21, cv2.VideoWriter_fourcc(*'XVID'),frate, (width, height))

print("INFO: Tracking is in Progress...")
frame_count = 0
start = time.time()
dist_dict = {}
first_frame = True
while True:

    ret, frame = cap.read()  # import image
    frame_count +=  1
    if frame_count == 5: # 3600
      break

    if  ret:  # if there is a frame continue with code
        if first_frame:
            imgRef = frame
            first_frame = False

        frame = alignImages(frame, imgRef)
        image, image2 = rotate_image(frame, 1.58)
        '''
        cropped_image = region_of_interest(image)
        canny_image = canny_edge_detector(cropped_image)
        minLineLength = 100
        maxLineGap = 50
        lines = cv2.HoughLinesP(canny_image,1,np.pi/180,100,minLineLength,maxLineGap)
        for x1,y1,x2,y2 in lines[0]:
            #cv2.line(image,(x1,y1),(x2,y2),(0,0,255),2)
            dist_dict[(x1,x2)] = (y1,y2)
        image1 = getTransform(image, y2) 
        #image = shadow_removal(image)
        #canny_image = canny_edge_detector(image1)
        '''
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # converts image to gray
        fg_mask = knnSubtractor.apply(gray)  #  mog2Subtractor  knnSubtractor
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        value1, th = cv2.threshold(fg_mask, 5, 255, cv2.THRESH_BINARY)
        opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=1) #erosion followed by dilation, useful in removing noises
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=3) #Dilation followed by Erosion
        dilation = cv2.dilate(closing, kernel, iterations = 1)
        '''        
        dilation = cv2.dilate(opening, kernel, iterations=2) #adds pixels to the boundaries of objects in an image
        #eroded = cv2.erode(dilation, kernel, iterations=1 ) #removes pixels on object boundaries        
        '''
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        minarea = 700
        # max area for contours, can be quite large for buses
        maxarea = 19000
        # vectors for the x and y locations of contour centroids in current frame
        '''cxx = np.zeros(len(contours))
        cyy = np.zeros(len(contours))
        '''
        for i in range(len(contours)):  # cycles through all contours in current frame
            if hierarchy[0, i, 3] == -1:  # using hierarchy to only count parent contours (contours not within others)
                area = cv2.contourArea(contours[i])  # area of contour
                if minarea < area < maxarea:  # area threshold for contour
                    # calculating centroids of contours
                    cnt = contours[i]
                    M = cv2.moments(cnt)
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                    
                    x, y, w, h = cv2.boundingRect(cnt)
                    
                    xA = x
                    xB = x + w
                    yA = y
                    yB = y + h
                    if w  > 300 or w < 50 or h < 22 or h > 90:
                        continue
                    if w > 300 and h > 90:
                        continue
                        
                    not_matched = True
                    for idx, centroid_data in enumerate(centroids_list):
                        if centroid_data[0] == count:
                            continue
                        if centroids_list[idx][4] == 0:
                            centroids_list[idx][5] = "unlocked"
                            centroids_list[idx][4] = 5
            
                        # check proximity using manhattan distance
                        X = abs(float(centroid_data[2][0] + centroid_data[2][2]) / 2 - float(xA + xB) / 2)
                        Y = abs(float(centroid_data[2][1] + centroid_data[2][3]) / 2 - float(yA + yB) / 2)

                        if X < 50 and Y < 15:
                            if h < 40 and X < 35:            
                                not_matched = False
                                centroids_list[idx][4] = 5
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                if centroids_list[idx][5] == "unlocked":
                
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                
                                    else:
                                        centroids_list[idx][3] = 0
                
                                if centroids_list[idx][3] == 3:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                if centroids_list[idx][6][-1] != 0.0:
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                    cv2.putText(image, str(int(centroids_list[idx][6][-1]))+ str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                    cv2.putText(image, str("bike"),(centroid_data[2][0]+100, centroid_data[2][1]+20) ,
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,20,147), 2, cv2.LINE_AA)
                            elif h > 40 and w > 185:          
                                not_matched = False
                                centroids_list[idx][4] = 5
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                #if cx < 300 and cx > 3700:
                                if centroids_list[idx][5] == "unlocked":
                
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                
                                    else:
                                        centroids_list[idx][3] = 0
            
                                #if cx > 300 and cx < 3700:
                                if centroids_list[idx][3] == 3:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                if centroids_list[idx][6][-1] != 0.0:
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                    cv2.putText(image, str(int(centroids_list[idx][6][-1]))+ str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                    cv2.putText(image, str("Bus"), (centroid_data[2][0], centroid_data[2][1]+40),
                                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,20,147), 2, cv2.LINE_AA)
                            else:          
                                not_matched = False
                                centroids_list[idx][4] = 5
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                if centroids_list[idx][5] == "unlocked":
                
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                
                                    else:
                                        centroids_list[idx][3] = 0
                
                                if centroids_list[idx][3] == 3:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                if centroids_list[idx][6][-1] != 0.0:
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                    cv2.putText(image, str(int(centroids_list[idx][6][-1]))+ str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                    cv2.putText(image, str("Car"), (centroid_data[2][0], centroid_data[2][1]+40), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,20,147), 2, cv2.LINE_AA)
                                
                            centroids_list[idx][0] = count
                            break
            
                    if not_matched:
                        color = make_new_color()
            
                        # append new rectangle in previous cars list
                        car_count = random.randint(1, 40) 
                        centroids_list.appendleft([count, color, (xA, yA, xB, yB), 1, 5, "unlocked", [0], car_count])
                        #car_count += 1
                        # cv2.rectangle(image, (xA, yA), (xB, yB), color, 2)
                        # cv2.putText(image, "0",
                        #             (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
                        prev_color = color
                        prev_coords = [xA, yA, xB, yB]
            
                    # plot all remaining locked rectangles
                    for idx, centroid_data in enumerate(centroids_list):
                
                        if centroid_data[5] == "locked" and centroid_data[0] != count:
                            centroids_list[idx][4] -= 1
                            if centroids_list[idx][6][-1] != 0.0:
                                cv2.rectangle(image, (centroid_data[2][0], centroid_data[2][1]), (centroid_data[2][2], centroid_data[2][3]),
                                              centroid_data[1], 2)

                                cv2.putText(image, str(int(centroids_list[idx][6][-1])) + str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                            (centroid_data[2][0], centroid_data[2][1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                
                            if centroids_list[idx][4] == 0:
                              centroids_list[idx][5] = "unlocked"
                              centroids_list[idx][4] = 5
                              centroids_list[idx][3] = 0
                                
                        
                    centroids_list = deque([car_data for car_data in list(centroids_list) if count - car_data[0] < 20])
                    
     
    image[500:970 ,] = image2[500:970, ] ;image[1200:1600 ,] = image2[1200:1600 ,]

    image = im.resize(image, 2000)
    writer.write(image)
    '''cv2.imshow('output', closing)
    # Wait for Esc key to stop
    if cv2.waitKey(33) == 27:
        break '''
    # outputs all the video frames into out folder present in the working directory
    #cv2.imwrite("/content/drive/My Drive/Colab Notebooks/out/" + str(count) + ".jpg", image)
    count += 1

Time_Elapsed = time.time()-start
print("Time Elapsed=",Time_Elapsed,'s')
print("finished Successfully")
print(image.shape)
cap.release()
writer.release()
cv2.destroyAllWindows()
# Output: https://drive.google.com/drive/folders/12HLHqqccQvQoCJFofr83Kava4-CcVcK3 


ModuleNotFoundError: ignored

In [ ]:
import sys
sys.path.insert(0, "/content/drive/My Drive/Colab Notebooks/")
from align import alignImages

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Locations: /content/drive/My Drive/Colab Notebooks/
# -*- coding: utf-8 -*-
"""
Created on Mon June 1 22:47:29 2020
Updated on Mon June 12
@author: Kumar Gaurav
Institutions: EE, IIT Kanpur

Title : Traffic Vehicle Speed Estimation
------------------------------------------------------------------
calculates speed of vehicles in realtime in KMPH

key_formula =  ( speed in object plane / distance of camera from road ) = ( speed in image plane / focal length )
approx fps of the video = 30

to obtain accurate value distance to object (mm)  =   focal length (mm) * real height of the object (mm) * image height (pixels)
                                                       ----------------------------------------------------------------
                                                                 object height (pixels) * sensor height (mm)
Default Code
Updated 29 Jun 20
"""
import cv2
from google.colab.patches import cv2_imshow # Colab cv2 imshow
import numpy as np
import random
from collections import deque
import imutils as im
import time


def make_new_color():
    return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
def make_new_veh():
    return [random.randint(1, 50)]
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    result = result[:,50:4000]
    result1 = result.copy()
    result[:985 ,] = 0 ;result[1170: ,] = 0
    return result, result1
def shadow_removal(img):
    rgb_planes = cv2.split(img)
    
    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((5,5), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)
    
    result = cv2.merge(result_planes)
    result_norm = cv2.merge(result_norm_planes)
    return result_norm

# Sensor Details:
GSD = 0.0366 #3.66 cm/pixels
Iw = 4096; Sw = 13.2 # in mm
value = (GSD*Iw)/Sw
f = 8.8; distance = 100 # approximated, real distance = (R/r)*(f*W/w)
ratio = distance/f
count = 0

#/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4
# capt.Video frames from a video
print("INFO: Video Starting")
path = '/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4'
cap = cv2.VideoCapture(path)

print("Video Open-",cap.isOpened())
# Subtractors
sub = cv2.createBackgroundSubtractorMOG2()  # create background subtractor
mogSubtractor = cv2.bgsegm.createBackgroundSubtractorMOG(300)
mog2Subtractor = cv2.createBackgroundSubtractorMOG2() 
gmgSubtractor = cv2.bgsegm.createBackgroundSubtractorGMG(10, .8)
knnSubtractor = cv2.createBackgroundSubtractorKNN()   #Very efficient if number of foreground pixels is low
cntSubtractor = cv2.bgsegm.createBackgroundSubtractorCNT(5, True)
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,12))

centroids_list = deque([])
car_count = 0
'''with open("/content/drive/My Drive/Colab Notebooks/out1.csv", "w") as o:
    pass'''

#width = 1500 ; height = 820
width = 2000; height = 1093
#frate = int(cap.get(cv2.CAP_PROP_FPS))
frate = 10
#***************************************************************************************************************
path21 = '/content/drive/My Drive/Colab Notebooks/Videos/_Track_KNN_final4.mp4' # Video write
writer = cv2.VideoWriter(path21, cv2.VideoWriter_fourcc(*'XVID'),frate, (width, height))

print("INFO: Tracking is in Progress...")
frame_count = 0
start = time.time()

while True:

    ret, frame = cap.read()  # import image
    frame_count +=  1
    if frame_count == 3600:
      break


    if  ret:  # if there is a frame continue with code
        image, image2 = rotate_image(frame, 1.78)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # converts image to gray
        #image1 = shadow_removal(gray)
        fg_mask = knnSubtractor.apply(gray)  #  mog2Subtractor  knnSubtractor
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        value, th = cv2.threshold(fg_mask, 10, 255, cv2.THRESH_BINARY)
        opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=1) #erosion followed by dilation, useful in removing noises
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=3) #Dilation followed by Erosion
        dilation = cv2.dilate(closing, kernel, iterations = 1)
        '''        
        dilation = cv2.dilate(opening, kernel, iterations=2) #adds pixels to the boundaries of objects in an image
        #eroded = cv2.erode(dilation, kernel, iterations=1 ) #removes pixels on object boundaries        
        '''
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        minarea = 700
        # max area for contours, can be quite large for buses
        maxarea = 18000
        # vectors for the x and y locations of contour centroids in current frame
        '''cxx = np.zeros(len(contours))
        cyy = np.zeros(len(contours))
        '''
        for i in range(len(contours)):  # cycles through all contours in current frame
            if hierarchy[0, i, 3] == -1:  # using hierarchy to only count parent contours (contours not within others)
                area = cv2.contourArea(contours[i])  # area of contour
                if minarea < area < maxarea:  # area threshold for contour
                    # calculating centroids of contours
                    cnt = contours[i]
                    M = cv2.moments(cnt)
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                    
                    x, y, w, h = cv2.boundingRect(cnt)
                    
                    xA = x
                    xB = x + w
                    yA = y
                    yB = y + h
                    if w  > 300 or w < 50:
                        continue
                    elif h < 20 or h > 90:
                        continue
                    elif w > 300 and h > 90:
                        continue
                        
                    not_matched = True
                    for idx, centroid_data in enumerate(centroids_list):
                        if centroid_data[0] == count:
                            continue
                        if centroids_list[idx][4] == 0:
                            centroids_list[idx][5] = "unlocked"
                            centroids_list[idx][4] = 5
            
                        # check proximity using manhattan distance
                        X = abs(float(centroid_data[2][0] + centroid_data[2][2]) / 2 - float(xA + xB) / 2)
                        Y = abs(float(centroid_data[2][1] + centroid_data[2][3]) / 2 - float(yA + yB) / 2)

                        if X < 50 and Y < 3:
                            if h < 40 and X < 35:            
                                not_matched = False
                                centroids_list[idx][4] = 5
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                if centroids_list[idx][5] == "unlocked":
                
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                
                                    else:
                                        centroids_list[idx][3] = 0
                
                                if centroids_list[idx][3] == 3:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                if centroids_list[idx][6][-1] != 0.0:
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                    cv2.putText(image, str(int(centroids_list[idx][6][-1]))+ str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                    cv2.putText(image, str("bike"),(centroid_data[2][0]+100, centroid_data[2][1]+20) ,
                                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,20,147), 2, cv2.LINE_AA)
                            elif h > 40 and w > 200:          
                                not_matched = False
                                centroids_list[idx][4] = 5
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                #if cx < 300 and cx > 3700:
                                if centroids_list[idx][5] == "unlocked":
                
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                
                                    else:
                                        centroids_list[idx][3] = 0
            
                                #if cx > 300 and cx < 3700:
                                if centroids_list[idx][3] == 3:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                if centroids_list[idx][6][-1] != 0.0:
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                    cv2.putText(image, str(int(centroids_list[idx][6][-1]))+ str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                    cv2.putText(image, str("Bus"), (centroid_data[2][0], centroid_data[2][1]+40),
                                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,20,147), 2, cv2.LINE_AA)
                            else:          
                                not_matched = False
                                centroids_list[idx][4] = 5
                                centroids_list[idx][2] = [xA, yA, xB, yB]
                                #*********************************************************************************************************
                                #centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*value*3.6/100/2)
                                centroids_list[idx][6].append(np.sqrt(X ** 2 + Y ** 2) * 60*3.6*ratio/100/2)
                                if centroids_list[idx][5] == "unlocked":
                
                                    if centroids_list[idx][0] == count - 1:
                                        centroids_list[idx][3] += 1
                
                                    else:
                                        centroids_list[idx][3] = 0
                
                                if centroids_list[idx][3] == 3:
                                    centroids_list[idx][5] = "locked"
                                    centroids_list[idx][3] = 0
                                if centroids_list[idx][6][-1] != 0.0:
                                    cv2.rectangle(image, (xA, yA), (xB, yB), centroid_data[1], 2)
                                    cv2.putText(image, str(int(centroids_list[idx][6][-1]))+ str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                                (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                    cv2.putText(image, str("Car"), (centroid_data[2][0], centroid_data[2][1]+40), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,20,147), 2, cv2.LINE_AA)
                                
                            centroids_list[idx][0] = count
                            break
            
                    if not_matched:
                        color = make_new_color()
            
                        # append new rectangle in previous cars list
                        car_count = random.randint(1, 40) 
                        centroids_list.appendleft([count, color, (xA, yA, xB, yB), 1, 5, "unlocked", [0], car_count])
                        #car_count += 1
                        # cv2.rectangle(image, (xA, yA), (xB, yB), color, 2)
                        # cv2.putText(image, "0",
                        #             (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
                        prev_color = color
                        prev_coords = [xA, yA, xB, yB]
            
                    # plot all remaining locked rectangles
                    for idx, centroid_data in enumerate(centroids_list):
                
                        if centroid_data[5] == "locked" and centroid_data[0] != count:
                            centroids_list[idx][4] -= 1
                            if centroids_list[idx][6][-1] != 0.0:
                                cv2.rectangle(image, (centroid_data[2][0], centroid_data[2][1]), (centroid_data[2][2], centroid_data[2][3]),
                                              centroid_data[1], 2)

                                cv2.putText(image, str(int(centroids_list[idx][6][-1])) + str("KMPH") + str(",ID=") + str(centroids_list[idx][7]),
                                            (centroid_data[2][0], centroid_data[2][1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (np.average(centroid_data[1])/2), 1, cv2.LINE_AA) 
                                
                            if centroids_list[idx][4] == 0:
                              centroids_list[idx][5] = "unlocked"
                              centroids_list[idx][4] = 5
                              centroids_list[idx][3] = 0
                                
                        if count - centroid_data[0] == 20:
                            if sum(centroid_data[6]) / len(centroid_data[6]) != 0.0:
                                with open("out1.csv", "a") as o:
                                    o.write(str(centroid_data[7]) + ": " + str(sum(centroid_data[6]) / len(centroid_data[6])) + "\n")
                
                    centroids_list = deque([car_data for car_data in list(centroids_list) if count - car_data[0] < 20])
                    
     
    image[500:985 ,] =image2[500:985, ] ;image[1170:1600 ,] = image2[1170:1600 ,]

    image = im.resize(image, 2000)
    writer.write(image)
    '''cv2.imshow('output', closing)
    # Wait for Esc key to stop
    if cv2.waitKey(33) == 27:
        break '''
    # outputs all the video frames into out folder present in the working directory
    #cv2.imwrite("/content/drive/My Drive/Colab Notebooks/out/" + str(count) + ".jpg", image)
    count += 1

Time_Elapsed = time.time()-start
print("Time Elapsed=",Time_Elapsed,'s')
print("finished Successfully")
print(image.shape)
cap.release()
writer.release()
cv2.destroyAllWindows()
# Output: https://drive.google.com/drive/folders/12HLHqqccQvQoCJFofr83Kava4-CcVcK3 


INFO: Video Starting
Video Open- False
INFO: Tracking is in Progress...


NameError: ignored

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul  5 07:42:04 2020
updated on July 12
@author: KGaurav
Title: Real Time Road Lane Detection [with image alignment]
minLineLength = 100
maxLineGap = 140
"""
import numpy as np
import cv2
import os
import sympy as sy
import sys
sys.path.insert(0, "/content/drive/My Drive/Colab Notebooks/")
from linearRegresion import regression

cv2.destroyAllWindows()

def clipVideo(image):
    img = np.zeros(image.shape, np.uint8)
    img2 = None
    #We could have used fixed numbers as the vertices of the polygon,
    #but they will not be applicable to images with different dimesnions.
    rows, cols = image.shape[:2] # (hight, width)
    h1 = 864; h2 = 1296

    img[h1:h2, ] = image[h1:h2, ] # image[height_range, width_range]
    #print(img.shape)
    return img, img2

def RGB_color_selection(image):

    #White color mask
    lower_threshold = np.uint8([200, 200, 200])
    upper_threshold = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(image, lower_threshold, upper_threshold)
    
    masked_image = cv2.bitwise_and(image, image, mask = white_mask)
    
    return masked_image

def convert_hsv(image):

    return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


def HSV_color_selection(image):

    #Convert the input image to HSV
    converted_image = convert_hsv(image)
    
    #White color mask
    lower_threshold = np.uint8([0, 0, 210])
    upper_threshold = np.uint8([255, 30, 255])
    white_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)

    #Combine white and yellow masks
    #mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = white_mask)
    
    return masked_image

def convert_hsl(image):

    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)


def HSL_color_selection(image):

    #Convert the input image to HSL
    converted_image = convert_hsl(image)
    
    #White color mask
    lower_threshold = np.uint8([0, 200, 0])
    upper_threshold = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)

    #Combine white and yellow masks
    #mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = white_mask)
    
    return masked_image

def gray_scale(image):

    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def gaussian_smoothing(image, kernel_size = 13):

    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

def canny_detector(image, low_threshold = 50, high_threshold = 150):

    return cv2.Canny(image, low_threshold, high_threshold)

def region_selection(image):

    mask = np.zeros_like(image)   
    #Defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #We could have used fixed numbers as the vertices of the polygon,
    #but they will not be applicable to images with different dimesnions.
    rows, cols = image.shape[:2]
    #print(rows, cols, image.shape)
    top_left     = [cols * 0.0, rows * 0.453]   # 950
    bottom_left  = [cols * 0.0, rows * 0.495]   # 1075 (width, height)
    bottom_right = [cols * 1.0, rows * 0.546]   # 1190 0r 300
    top_right    = [cols * 1.0, rows * 0.500]   # 1060
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def hough_transform(image):
    """
    Determine and cut the region of interest in the input image.
        Parameters:
            image: The output of a Canny transform.
    """
    rho = 1              #Distance resolution of the accumulator in pixels.
    theta = np.pi/180    #Angle resolution of the accumulator in radians.
    threshold = 20       #Only lines that are greater than threshold will be returned.
    minLineLength = 50   #Line segments shorter than that are rejected.
    maxLineGap = 125     #Maximum allowed gap between points on the same line to link them
    return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold,
                           minLineLength = minLineLength, maxLineGap = maxLineGap)


def average_slope_intercept(lines):
    """
    Find the slope and intercept of the left and right lanes of each image.
        Parameters:
            lines: The output lines from Hough Transform.
    """
    left_lines    = [] #(slope, intercept)
    left_weights  = [] #(length,)
    right_lines   = [] #(slope, intercept)
    right_weights = [] #(length,)
    #print("length of points before=",len(lines))
    idx = -1
    slopeIntercept = []
    for idx1, line in enumerate(lines):
        #print(line)
        for x1, y1, x2, y2 in line:
            length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))

            if y1 > y2 or length > 150 or x1 == x2 or (y2-y1)>10:
                continue
            if idx1 > 0:
                if (lines[idx1][0][1] - lines[idx1-1][0][3]) > 10: #(Y2 - Y1)
                    continue
                if (lines[idx1][0][1] - lines[idx1-1][0][3]) > 150:
                    continue
            '''param = np.polyfit((x1, x2), (y1, y2), 1) # (slope, intercept)=(param[0], param[1])
            
            if param[0] > 0.045:
                continue '''

            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - (slope * x1)
            if slope < 0 or slope == 0:
                continue
            else:
                '''
                right_lines.append([x1,y1,x2,y2])
                #print(right_lines)
                if  idx > 0:
                    if abs(right_lines[idx][0] - right_lines[idx-1][0]) < 40:
                        right_lines.pop(idx)
                        idx -= 1
                    param1 = np.polyfit((right_lines[idx-1][2], right_lines[idx][0]), (right_lines[idx-1][3], lines[idx][0][1]), 1)
                    if param1[0] > 0.045 or param1[0] < 0.005:
                        continue
                    print("slope=",param1[0])
                    print("intercept=",int(intercept),"slope= %.3f" % slope,"length=",int(length),right_lines[idx],(y2-y1) )
            idx += 1'''
                if idx1 > 0:
                    '''if abs(lines[idx1][0][0] - lines[idx1-1][0][0]) < 100:
                        continue'''
                    '''param1 = np.polyfit((lines[idx1-1][0][2], lines[idx1][0][0]), (lines[idx1-1][0][3], lines[idx1][0][1]), 1)
                    if param1[0] > 0.035 or param1[0] < 0.005:
                        continue  '''
                    right_lines.append([x1,y1,x2,y2])
                    #print("intercept=",int(intercept),"slope= %.3f" % slope,"length=",int(length),right_lines[idx],(y2-y1) )
                else:
                    right_lines.append([x1,y1,x2,y2])

    return right_lines

def pixel_points(y1, y2, line):

    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

def lane_lines(image, lines):
    right_lane = average_slope_intercept(lines)

    return right_lane

    
def draw_lane_lines(image, lines, color=[0, 0, 255], thickness=6):

    line_image = np.zeros_like(image)
    #print(lines)
    if lines is not None: 
        '''for [x1, y1, x2, y2] in lines: 
            cv2.line(line_image, (x1, y1), (x2, y2), color, thickness)   '''
         
        for idx, line in enumerate(lines): 

            cv2.line(line_image, (lines[idx][0], lines[idx][1]), (lines[idx][2], lines[idx][3]), color, thickness) 
            if idx > 0:
                '''if lines[idx][3] - lines[idx-1][3] > 25:
                    continue  '''
                cv2.line(line_image, (lines[idx-1][2], lines[idx-1][3]), (lines[idx][0], lines[idx][1]), color, thickness) 

    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)
             
def align_image(img, S1, I1, S0, I0):
    rows,cols = img.shape[:2]
    #Shift in Y
    Ty = -(I1-I0) ; Tx = 0
    #Shift in X
    #Tx = ((I1/S1)-(I0/S0))
    Tx = 0
    theta = -(S1 - S0)
    #print("Tx, Ty =",Tx,Ty, sy.cos(theta))
    M = np.float32([[sy.cos(theta),sy.sin(theta),0],[sy.sin(theta),sy.cos(theta),Ty]])
    dst = cv2.warpAffine(img,M,(cols,rows))
    return dst
    
if __name__ == '__main__':

    # Path of dataset directory 
    path = '/content/drive/My Drive/Colab Notebooks/Videos/1stDrone.MP4'
    #path = "E:\CodeVideoData\ClipDroneData\road_lane.mp4"
    cap = cv2.VideoCapture(path)  
    print("Video Status: ",cap.isOpened())
    #cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    frame_count = 0
    dist_dict = {}
    frame_count = -1
    #width = 1500; height = 820
    width = 4096; height = 2160
    #width = 4096; height = 432
    #frate = int(cap.get(cv2.CAP_PROP_FPS))
    frate = 30
    #***************************************************************************************************************
    path21 = '/content/drive/My Drive/Colab Notebooks/Colab Notebooks Anshu/Videos/_Track_laneLine_alignVideo_01.mp4' # Video write
    writer = cv2.VideoWriter(path21, cv2.VideoWriter_fourcc(*'XVID'),frate, (width, height))
    slopeIntercept = []
    first_frame = True
    while(cap.isOpened()):
        ret, frame = cap.read()
        frame_count += 1
        if frame_count == 3600: # 3600
          break

        clip, img2   = clipVideo(frame)
        color_select = HSL_color_selection(clip)
        gray         = gray_scale(color_select)
        smooth       = gaussian_smoothing(gray)
        edges        = canny_detector(smooth)
        region       = region_selection(edges)
        hough        = hough_transform(region)

        hough = hough.reshape(int(2*len(hough)),2)
        hough.sort(axis=0) 
        hough = hough.reshape(len(hough)//2,1,4)
        
        #print("Number of points=",len(hough))
        laneLines   = lane_lines(clip, hough)
        #print("length of points after",len(laneLines))
        laneLines = np.array(laneLines)
        laneLines = laneLines.reshape(int(2*len(laneLines)),2)
        if first_frame:
            slope_1st, intercept_1st = regression(laneLines)
            alignImage = align_image(frame, slope_1st, intercept_1st,slope_1st, intercept_1st)
            first_frame = False
            #print("Parameters=",slope_1st, intercept_1st)
        else:
            slope, intercept = regression(laneLines)
            alignImage = align_image(frame, slope, intercept, slope_1st, intercept_1st)

        slopeIntercept = []

        writer.write(frame)

  
    print("Finished")
    # destroy all the windows that is currently on 
    print("Frame Size=",frame.shape)
    cap.release() 
    writer.release()
    cv2.destroyAllWindows()  
    

Video Status:  True
Finished
Frame Size= (2160, 4096, 3)


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 15 16:28:37 2020
updated on July 15
@author: KGaurav
Title : Align Drone Video Frames
Directory: Work
"""

import cv2
import numpy as np
import os
sys.path.insert(0, os.path.abspath("/content/drive/My Drive/Colab Notebooks/"))
from align import alignImages

# Path of dataset directory 
#path = "E:\CodeVideoData\_DroneData\ClipDroneData\_1stDrone.MP4"  # _1stDrone_2~2
print("INFO: Video Starting")
path = '/content/drive/My Drive/Colab Notebooks/Videos/3rdDrone.MP4'  # 1stDrone DJI_Morning
cap = cv2.VideoCapture(path)  
print("Video Status: ",cap.isOpened())

#cv2.namedWindow("output", cv2.WINDOW_NORMAL)
frame_count = 0

frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT ))
#width = 1500; height = 820
width = 4096; height = 2160
#width = 4096; height = 432
frate = int(cap.get(cv2.CAP_PROP_FPS))
#frate = 10
#***************************************************************************************************************
path21 = '/content/drive/My Drive/Colab Notebooks/Videos/_1stDrone_aligned.mp4' # Video write
writer = cv2.VideoWriter(path21, cv2.VideoWriter_fourcc(*'XVID'),frate, (width, height))
first_frame = True
print("INFO: Aligning is in Progress...",frames)

while(cap.isOpened()):
    ret, frame = cap.read()
    frame_count += 1
    if frame_count == 3660: # 10800,  3663
      break
    if first_frame:
        imgRef = frame
        first_frame = False
    image = alignImages(frame , imgRef)
    #result[0:864 ,] = frame[0:864, ] ;result[1296: ,] = frame[1296: ,]
    #print("frame count = ",frame_count)
    #cv2.imwrite("out/" + str(frame_count) + ".jpg",alignImage)
    #cv2.imwrite("houghlines_result.jpg",frame)
    writer.write(image)

  
# destroy all the windows that is currently on 
print("Frame Size=",image.shape)
cap.release() 
writer.release()
cv2.destroyAllWindows()  



INFO: Video Starting
Video Status:  True
INFO: Aligning is in Progress... 3669
Frame Size= (2160, 4096, 3)
